# CALIDAD DE DATOS
## Dataset: Water Potability

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
from sklearn.impute import SimpleImputer
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('water_potability.csv')
df.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3276 entries, 0 to 3275
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ph               2785 non-null   float64
 1   Hardness         3276 non-null   float64
 2   Solids           3276 non-null   float64
 3   Chloramines      3276 non-null   float64
 4   Sulfate          2495 non-null   float64
 5   Conductivity     3276 non-null   float64
 6   Organic_carbon   3276 non-null   float64
 7   Trihalomethanes  3114 non-null   float64
 8   Turbidity        3276 non-null   float64
 9   Potability       3276 non-null   int64  
dtypes: float64(9), int64(1)
memory usage: 256.1 KB


# A. PERFILADO DE DATOS

In [6]:
profile = ProfileReport(df, title="Water Potability - Data Quality Report")
profile.to_file("water_potability_profiling_report.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 65.34it/s]


# B. DIAGNÓSTICO DE CALIDAD DE DATOS

**Dimensiones a evaluar:**
- **Completitud:** ¿Hay datos faltantes?
- **Validez:** ¿Los valores están en rangos válidos?
- **Precisión:** ¿Existen outliers?
- **Consistencia:** ¿Las correlaciones son lógicas?
- **Unicidad:** ¿Existen duplicados?

In [7]:
# 1. Completitud - Valores faltantes
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100
print("Valores faltantes por variable:")
print(pd.DataFrame({'Faltantes': missing_values, 'Porcentaje (%)': missing_percentage})[missing_values > 0])

Valores faltantes por variable:
                 Faltantes  Porcentaje (%)
ph                     491       14.987790
Sulfate                781       23.840049
Trihalomethanes        162        4.945055


In [8]:
# 2. Validez - Rangos esperados y valores negativos
print("Valores negativos por variable:")
for col in df.select_dtypes(include=[np.number]).columns:
    negative_count = (df[col] < 0).sum()
    if negative_count > 0:
        print(f"  {col}: {negative_count}")
    else:
        print(f"  {col}: 0")

Valores negativos por variable:
  ph: 0
  Hardness: 0
  Solids: 0
  Chloramines: 0
  Sulfate: 0
  Conductivity: 0
  Organic_carbon: 0
  Trihalomethanes: 0
  Turbidity: 0
  Potability: 0


In [9]:
# 3. Precisión - Outliers por método IQR
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)][column]
    return len(outliers)

numeric_cols = df.select_dtypes(include=[np.number]).columns.drop('Potability')
print("Outliers detectados por variable:")
for col in numeric_cols:
    outliers = detect_outliers_iqr(df, col)
    pct = (outliers / len(df.dropna(subset=[col]))) * 100
    print(f"  {col}: {outliers} ({pct:.2f}%)")

Outliers detectados por variable:
  ph: 46 (1.65%)
  Hardness: 83 (2.53%)
  Solids: 47 (1.43%)
  Chloramines: 61 (1.86%)
  Sulfate: 41 (1.64%)
  Conductivity: 11 (0.34%)
  Organic_carbon: 25 (0.76%)
  Trihalomethanes: 33 (1.06%)
  Turbidity: 19 (0.58%)


In [10]:
# 4. Consistencia - Correlaciones
df_num = df.copy()
correlation_matrix = df_num.corr()
print("Correlación con variable objetivo (Potability):")
print(correlation_matrix['Potability'].sort_values(ascending=False))

Correlación con variable objetivo (Potability):
Potability         1.000000
Solids             0.033743
Chloramines        0.023779
Trihalomethanes    0.007130
Turbidity          0.001581
ph                -0.003556
Conductivity      -0.008128
Hardness          -0.013837
Sulfate           -0.023577
Organic_carbon    -0.030001
Name: Potability, dtype: float64


In [11]:
# 5. Unicidad - Duplicados
duplicados = df.duplicated().sum()
print(f"Registros duplicados: {duplicados}")
print(f"Porcentaje: {(duplicados / len(df) * 100):.2f}%")

Registros duplicados: 0
Porcentaje: 0.00%


# Diagnóstico por dimensiones de calidad de datos

A partir del perfilado (`water_potability_profiling_report.html`) y de los análisis del notebook, el diagnóstico por dimensión es:

- Completitud:
  - Faltantes detectados: ph = 491 (14.99%), Sulfate = 781 (23.84%), Trihalomethanes = 162 (4.95%).
  - Impacto: la variable Sulfate presenta un nivel alto de faltantes que puede sesgar el entrenamiento si no se trata.
  - Acción: se imputa con la mediana para variables numéricas (implementado en la sección C. Limpieza), preservando la distribución y robusto a outliers.

- Validez:
  - No se detectaron valores negativos en variables numéricas (todas las variables reportan 0).
  - Observación: según el perfilado, los rangos observados son plausibles para las magnitudes medidas; no se evidencian valores físicamente imposibles.

- Precisión (Outliers):
  - Por método IQR, los porcentajes de outliers por variable están en el rango ~0.3% a ~2.5% (p. ej., Hardness ≈ 2.53%, Chloramines ≈ 1.86%).
  - Impacto: bajo a moderado; pueden afectar modelos sensibles a valores extremos.
  - Acción: winsorización al 1% en cada cola (implementado en la sección C) para atenuar extremos sin eliminar observaciones.

- Consistencia:
  - Correlación con la variable objetivo (Potability) es débil para todas las variables (valores cercanos a 0), lo que sugiere relaciones no lineales o efectos combinados.
  - Implicación: conviene evaluar modelos no lineales y con interacción/ensamble (como Random Forest, SVM, MLP), como se realiza en el notebook de minería.

- Unicidad:
  - Registros duplicados: 0 (0.00%).
  - Sin acciones requeridas.

Conclusión: tras imputación por mediana, winsorización al 1% y normalización, el dataset queda en condiciones adecuadas para modelado. Se recomienda mantener el reporte de perfilado junto a los datos como evidencia del estado de calidad.

# C. LIMPIEZA Y MEJORA DE DATOS

In [12]:
df_clean = df.copy()

# 1. Eliminar duplicados
df_clean = df_clean.drop_duplicates()

# 2. Imputación de valores faltantes por mediana
features = df_clean.drop('Potability', axis=1)
target = df_clean['Potability']

imputer = SimpleImputer(strategy='median')
features_imputed = pd.DataFrame(
    imputer.fit_transform(features),
    columns=features.columns,
    index=features.index
)
df_clean = pd.concat([features_imputed, target.reset_index(drop=True)], axis=1)

In [13]:
# 3. Tratamiento de outliers con Winsorización
features_winsorized = features_imputed.copy()

for col in features_winsorized.columns:
    features_winsorized[col] = winsorize(features_winsorized[col], limits=[0.01, 0.01])

df_clean = pd.concat([features_winsorized, target.reset_index(drop=True)], axis=1)

In [14]:
# 4. Normalización con StandardScaler
scaler = StandardScaler()
features_scaled = pd.DataFrame(
    scaler.fit_transform(features_winsorized),
    columns=features_winsorized.columns,
    index=features_winsorized.index
)

df_final = pd.concat([features_scaled, target.reset_index(drop=True)], axis=1)
df_final.describe()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
count,3.276000e+03,3.276000e+03,3.276000e+03,3.276000e+03,3.276000e+03,3.276000e+03,3.276000e+03,3.276000e+03,3.276000e+03,3276.000000
mean,-2.830459e-16,1.756836e-16,8.729959e-17,-3.426916e-16,-4.511382e-16,1.012350e-15,-6.105549e-16,-6.506802e-17,-3.361847e-16,0.390110
std,1.000153e+00,1.000153e+00,1.000153e+00,1.000153e+00,1.000153e+00,1.000153e+00,1.000153e+00,1.000153e+00,1.000153e+00,0.487849
min,-2.567135e+00,-2.567041e+00,-1.948376e+00,-2.549040e+00,-2.796085e+00,-1.969739e+00,-2.432843e+00,-2.469664e+00,-2.372070e+00,0.000000
25%,-5.626020e-01,-6.071516e-01,-7.349369e-01,-6.452571e-01,-4.741454e-01,-7.585094e-01,-6.819286e-01,-6.353240e-01,-6.873540e-01,0.000000
50%,-2.735821e-02,1.848534e-02,-1.230703e-01,5.823386e-03,-1.665532e-02,-5.292395e-02,-1.975635e-02,1.243925e-02,-1.493290e-02,0.000000
75%,5.602185e-01,6.311456e-01,6.218175e-01,6.450297e-01,4.790083e-01,6.998690e-01,6.998730e-01,6.647025e-01,6.966024e-01,1.000000
max,2.638764e+00,2.541107e+00,2.801798e+00,2.518076e+00,2.731109e+00,2.409985e+00,2.402369e+00,2.484052e+00,2.326732e+00,1.000000


In [ ]:
# 5. Guardar datos limpios
df_clean_final = pd.concat([features_winsorized, target.reset_index(drop=True)], axis=1)
df_clean_final.to_csv('water_potability_cleaned.csv', index=False) # Sin normalización
df_final.to_csv('water_potability_scaled.csv', index=False) # Con normalización